In [4]:
import pandas as pd
import numpy as np
import re
from utils.processing_data import is_IP

In [30]:
import pickle

with open('/Users/phamvanmanh/Documents/GitHub/NER_MOCKPROJECT/test_final.pkl', 'rb') as f:
    data = pickle.load(f)

In [90]:
import string
import unicodedata
from pyvi import ViTokenizer, ViPosTagger
import re


def preprocess_email_url(datas):
  datas_trained = []
  for i in range(len(datas)):
    data = datas[i]

    if data[1] == 'EMAIL':
      check = is_Email(data[0])
      if len(check) == 0:
        data = (data[0], 'O')
    
    if data[1] != 'EMAIL' and  data[1] != 'URL': #(url, org, loc, o,.....)
      check = is_Email(data[0])
      if len(check) > 0:
        data = (data[0], 'EMAIL')

  

    if data[1] == "URL":
      # print(data[0])
      check = is_URL(data[0])
      if len(check) > 0 and  check[0][1] - check[0][0] == len(data[0]):
        data = (data[0], 'URL')
      else: 
        data = (data[0], 'O')
      
    try:
      if data[1] != 'URL' and data[1] != 'EMAIL':
        check = is_URL(data[0])
        if len(check) > 0 and  check[0][1] - check[0][0] == len(data[0]):
          data = (data[0], 'URL')
    except:
      print(check)
    datas_trained.append(data)
  return datas_trained
    
# sent = 'pham van manh have email ( pvm26042000@gmail.com ) ....'
# out = [('pham', 'O'), ('van', 'O'), ('manh', 'O'), ('have', 'O'), ('email', 'O'), ('(', 'O'),  ('pvm26042000', 'EMAIL'), ('@', 'EMAIL'),('gmail', 'EMAIL'), ('.', 'EMAIL'),('com', 'EMAIL'),(')', 'O'),('....', 'O')]

def merge_word(sent, pred_out):
  '''
    :sent: is input sentences (hanlded pre-processing). example: 'pham van manh have email ( pvm26042000@gmail.com ) ....'
    :out : is input of predict, is list tuple. example: [('pham', 'O'), ('van', 'O'), ('manh', 'O'), ('have', 'O'), ('email', 'O'), ('(', 'O'),  ('pvm26042000', 'EMAIL'), ('@', 'EMAIL'),('gmail', 'EMAIL'), ('.', 'EMAIL'),('com', 'EMAIL'),(')', 'O'),('....', 'O')]
  '''
  out_merged = []
  parts = sent.split()
  for index in range(0, len(parts)):
    word = parts[index]

    
    for jndex in range(1, len(pred_out) + 1):
      token = pred_out[0:jndex]
      ws_token, _ = list(zip(*token))
      word_token = "".join(ws_token)
   
      if word_token == word:
        if len(token) == 1:
          out_merged.append(token[0])
        elif len(token) > 1:
          a, b = list(zip(*token))
          word_merged = "".join(a)
          l_merged = decide_label((word_merged, b))
          out_merged.append(l_merged)
        pred_out = pred_out[jndex:]
        break
  return out_merged

def post_processing(origin_sentence, out_predict):

  out_merged = merge_word(origin_sentence, out_predict)
  datas_trained = post_process_email_url(out_merged)
  
  gr_indexs = cluster(datas_trained, 3)
  print(gr_indexs)
  for index in gr_indexs:
    string, label = list(zip(*datas_trained[index[0]: index[-1] + 1]))

    if is_ADDRESS(string, label) == True:
      for i in range(index[0], index[-1] + 1):
        datas_trained[i] =(datas_trained[i][0], "ADDRESS")
    else:
      for i in range(index[0], index[-1] + 1):
        if datas_trained[i][0] == ',':
          datas_trained[i] = (datas_trained[i][0], "O")
        else:
          datas_trained[i] =(datas_trained[i][0], "LOCATION")
  return datas_trained

def cluster(data, maxgap):
    '''Arrange data into groups where successive elements
       differ by no more than *maxgap*
        >>> cluster([1, 6, 9, 100, 102, 105, 109, 134, 139], maxgap=10)
        [[1, 6, 9], [100, 102, 105, 109], [134, 139]]
        >>> cluster([1, 6, 9, 99, 100, 102, 105, 134, 139, 141], maxgap=10)
        [[1, 6, 9], [99, 100, 102, 105], [134, 139, 141]]
    '''

    black_list = [":", "(", ";", "{", "["]

    indexs = []
    for index in range(len(data)):
      token = data[index]
      if token[1] == "LOCATION" or token[1] == "ADDRESS" :
        indexs.append(index)

    indexs.sort()
    groups = [[indexs[0]]]

    for jndex in range(1,len(indexs[1:])):
      x  = indexs[jndex]
      # print(data[indexs[jndex-1]:x])
      w, labels = list(zip(*data[indexs[jndex-1]:x]))
      # print(any(character in w for character in black_list))
      if abs(x - groups[-1][-1]) <= maxgap and any(character in w for character in black_list) == False:
          groups[-1].append(x)
      elif any(character in data[indexs[jndex-1]:x] for character in black_list):
          groups.append([x])
      else:
          groups.append([x])
    return groups
  


def has_numbers(inputString):
  parts = inputString.split()
  for i in range(len(parts)):
    part = parts[i]
    for char in part:
      if char.isdigit():
        if i > 0 and parts[i-1].lower() in ["quận", "q."]:
          return False
        else:
          return True
  return False

def is_ADDRESS(string, label):

  uy_tin = 0
  string_loc = " ".join(string)

  level = ["số", "lô", "km","quốc_lộ","đại_lộ","kcn", "đường","tổ", "ngõ", "toà", "ngách", "hẻm","kiệt", "chung_cư", "số_nhà","ấp" ,"thôn", "khu","phố" , "quận", "phường", "xã", "thị_xã","huyện", "thành_phố", "tp", "tỉnh" ]
  level_0 ={'status': True,'keywords': ["toà", "chung_cư", "số", "lô", "số_nhà"] }
  level_1 = {'status': True, 'keywords': [ "ngõ", "ngách", "hẻm","kiệt","kcn", "km"]}
  level_2 = {'status': True, 'keywords':["ấp" ,"thôn", "khu","phố" , "quận", "phường", "xã", "tổ", "dân_phố", "đường", "quốc_lộ", "đại_lộ"]}
  level_3 = {'status': True,'keywords':["thị","huyện"]}
  level_4 = {'status': True,'keywords':["thành_phố", "tp", "tỉnh"]}

  parts =  ViPosTagger.postagging(ViTokenizer.tokenize(string_loc))[0]

  for index in range(len(parts)):
    seg_word = parts[index]
    if index == 0 and  has_numbers(seg_word.split(" ")[0]):
        uy_tin += 0.3
        break

    if seg_word.lower() in level:
 
      if seg_word.lower() in level_0['keywords'] and level_0['status'] == True:
        uy_tin += 0.3
        break

      elif seg_word.lower() in level_1['keywords'] and level_1['status'] == True:
        uy_tin += 0.25
        level_1['status'] = False

      elif seg_word.lower()  in level_2['keywords'] and level_2['status'] == True:
        uy_tin += 0.025
        level_2['status'] = False
      elif seg_word.lower() in  level_3['keywords'] and level_3['status'] == True:
   
        uy_tin += 0.015
        level_3['status'] = False
      elif seg_word.lower() in level_4['keywords'] and level_4['status'] == True:
     
        uy_tin += 0.01
        level_4['status'] = False
  print(uy_tin)
  if uy_tin >= 0.29:
    return True
  
  return False


def decide_label(part):
  word = part[0]
  labels = part[1]
  return (word, max(labels))


import re
def constain_alpha(token):

  for character in token:

    is_letter = character.isalpha()
    if is_letter == True:
      return True
  
  return False

def is_URL(token):
    token = token.lower()
    index = 0
    indexs = []
    if constain_alpha(token) == True:
      domain = re.findall(r'\b((?:https?://)?(?:(?:www\.)?(?:[\da-z\.-]+)\.(?:[a-z]{2,6})|(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)|(?:(?:[0-9a-fA-F]{1,4}:){7,7}[0-9a-fA-F]{1,4}|(?:[0-9a-fA-F]{1,4}:){1,7}:|(?:[0-9a-fA-F]{1,4}:){1,6}:[0-9a-fA-F]{1,4}|(?:[0-9a-fA-F]{1,4}:){1,5}(?::[0-9a-fA-F]{1,4}){1,2}|(?:[0-9a-fA-F]{1,4}:){1,4}(?::[0-9a-fA-F]{1,4}){1,3}|(?:[0-9a-fA-F]{1,4}:){1,3}(?::[0-9a-fA-F]{1,4}){1,4}|(?:[0-9a-fA-F]{1,4}:){1,2}(?::[0-9a-fA-F]{1,4}){1,5}|[0-9a-fA-F]{1,4}:(?:(?::[0-9a-fA-F]{1,4}){1,6})|:(?:(?::[0-9a-fA-F]{1,4}){1,7}|:)|fe80:(?::[0-9a-fA-F]{0,4}){0,4}%[0-9a-zA-Z]{1,}|::(?:ffff(?::0{1,4}){0,1}:){0,1}(?:(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])|(?:[0-9a-fA-F]{1,4}:){1,4}:(?:(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])))(?::[0-9]{1,4}|[1-5][0-9]{4}|6[0-4][0-9]{3}|65[0-4][0-9]{2}|655[0-2][0-9]|6553[0-5])?(?:/[\w\.-]*)*/?)\b', token)
      
      if len(domain) != 0:
          index_start_domain = token.find(domain[0]) + index
          if token.find(domain[0]) == 0:
              index_end_domain = index_start_domain + len(token)
          else:
              index_end_domain = index_start_domain + len(domain[0])
          indexs.append((index_start_domain, index_end_domain))
      index += len(token) + 1
    return indexs

def is_Email(token):
    index = 0
    indexs = []
    for word in token.split(" "):
        # print(word)
        emails = re.findall(r"[\w.+-]+@[\w-]+\.[\w.-]+", word)
        # print(emails)
        if len(emails) != 0:
            index_start_email = word.find(emails[0]) + index
            
            index_end_email = index_start_email + len(emails[0])
            
            indexs.append((index_start_email, index_end_email))
        index += len(word) + 1
    return indexs
def is_IP(token):
  index = 0
  indexs = []
  for word in token.split(" "):
      # print(word)
      emails = re.findall(r"\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b", word)
      # print(emails)
      if len(emails) != 0:
          index_start_email = word.find(emails[0]) + index
          
          index_end_email = index_start_email + len(emails[0])
          
          indexs.append((index_start_email, index_end_email))
      index += len(word) + 1
  return indexs

def post_process_email_url(datas):
  black_word = ["tp.hcm"]
  datas_trained = []
  for i in range(len(datas)):
    data = datas[i]

      # check predict email
    if data[1] == 'EMAIL':
        check = is_Email(data[0])
        if len(check) == 0:
          data = (data[0], 'O')
    
    elif data[1] == 'URL':
        check = is_URL(data[0])
        if len(check) == 0 or  check[0][1] - check[0][0]!= len(data[0]):
          data = (data[0], 'O')
    
    elif data[1] == 'IP':
        check = is_IP(data[0])
        if len(check) == 0 or  check[0][1] - check[0][0]!= len(data[0]):
          if data[0].isalnum():
            data = (data[0], 'QUANTITY')
          else:
            data = (data[0], 'O')

    if data[1] in ['O'] and data[1].lower() not in black_word:
        # print(data[0])
        check_url = is_URL(data[0])
        check_email= is_Email(data[0])

        if len(check_url) > 0 and  check_url[0][1] - check_url[0][0] == len(data[0]):

          data = (data[0], 'URL')

        elif len(check_email) > 0:
          data = (data[0], 'EMAIL')
      
    datas_trained.append(data)
  return datas_trained

In [52]:
data_add = []
for d in data:
    w, tr_n, pred_n, pos_n = list(zip(*d))
    if "ADDRESS" in tr_n:
        for index in range(len(tr_n)):
            if tr_n[index] != pos_n[index] and pos_n[index] == "ADDRESS":
                data_add.append(d)
                break
len(data_add)  


5

In [97]:
data_add[4]

[('Chung', 'O', 'O', 'O'),
 ('cư', 'O', 'O', 'O'),
 ('cao', 'O', 'O', 'O'),
 ('nhất', 'O', 'O', 'O'),
 (':', 'O', 'O', 'O'),
 ('Năm', 'DATETIME', 'DATETIME', 'DATETIME'),
 ('2014', 'DATETIME', 'DATETIME', 'DATETIME'),
 (',', 'O', 'O', 'O'),
 ('với', 'O', 'O', 'O'),
 ('việc', 'O', 'O', 'O'),
 ('sở', 'O', 'O', 'O'),
 ('hữu', 'O', 'O', 'O'),
 ('tòa', 'O', 'O', 'O'),
 ('nhà', 'O', 'O', 'O'),
 ('số', 'ADDRESS', 'ADDRESS', 'ADDRESS'),
 ('432', 'ADDRESS', 'ADDRESS', 'ADDRESS'),
 ('đại', 'ADDRESS', 'ADDRESS', 'ADDRESS'),
 ('lộ', 'ADDRESS', 'ADDRESS', 'ADDRESS'),
 ('Park', 'ADDRESS', 'ADDRESS', 'ADDRESS'),
 ('Avenue', 'ADDRESS', 'ADDRESS', 'ADDRESS'),
 (',', 'ADDRESS', 'ADDRESS', 'ADDRESS'),
 ('thành', 'ADDRESS', 'ADDRESS', 'ADDRESS'),
 ('phố', 'ADDRESS', 'ADDRESS', 'ADDRESS'),
 ('New', 'ADDRESS', 'ADDRESS', 'ADDRESS'),
 ('York', 'ADDRESS', 'ADDRESS', 'ADDRESS'),
 ('(', 'O', 'O', 'ADDRESS'),
 ('Mỹ', 'LOCATION', 'LOCATION', 'ADDRESS'),
 (')', 'O', 'O', 'O'),
 ('vượt', 'O', 'O', 'O'),
 ('qua', 'O

In [93]:
w, true, pred, pos_ner = list(zip(*data_add[3]))

In [94]:
pred_out = list(zip(w, pred))

In [95]:
post_processing(" ".join(w), pred_out)

[[4, 5, 6, 7, 8, 9, 10, 11, 12, 13], [15, 16, 17, 18, 19, 20, 21, 22, 23], [25, 26, 27, 28, 29, 30, 31, 32, 33]]
0.3
0.3
0.3


[('Ngoài', 'O'),
 ('ra', 'O'),
 (',', 'O'),
 ('tại', 'O'),
 ('463', 'ADDRESS'),
 ('Trần', 'ADDRESS'),
 ('Hưng', 'ADDRESS'),
 ('Đạo', 'ADDRESS'),
 (',', 'ADDRESS'),
 ('Dĩ', 'ADDRESS'),
 ('An', 'ADDRESS'),
 (',', 'ADDRESS'),
 ('Bình', 'ADDRESS'),
 ('Dương', 'ADDRESS'),
 (';', 'O'),
 ('63', 'ADDRESS'),
 ('Hoàng', 'ADDRESS'),
 ('Cầu', 'ADDRESS'),
 (',', 'ADDRESS'),
 ('Đống', 'ADDRESS'),
 ('Đa', 'ADDRESS'),
 (',', 'ADDRESS'),
 ('Hà', 'ADDRESS'),
 ('Nội', 'ADDRESS'),
 (';', 'O'),
 ('24', 'ADDRESS'),
 ('Lý', 'ADDRESS'),
 ('Thường', 'ADDRESS'),
 ('Kiệt', 'ADDRESS'),
 (',', 'ADDRESS'),
 ('Huế', 'ADDRESS'),
 (',', 'ADDRESS'),
 ('Thừa', 'ADDRESS'),
 ('Thiên', 'ADDRESS'),
 ('Huế', 'ADDRESS'),
 (';', 'O'),
 ('Trung', 'ORGANIZATION'),
 ('tâm', 'ORGANIZATION'),
 ('Giao', 'ORGANIZATION'),
 ('lưu', 'ORGANIZATION'),
 ('văn', 'ORGANIZATION'),
 ('hóa', 'ORGANIZATION'),
 ('phố', 'ORGANIZATION'),
 ('cổ', 'ORGANIZATION'),
 ('Hà', 'ORGANIZATION'),
 ('Nội', 'ORGANIZATION'),
 ('tổ', 'O'),
 ('chức', 'O'),
 ('trư

In [48]:
a = [1,2,3]
b = [1,2,5]
a ==b

False

In [50]:
# 6, 15 # dev
# 8, 10
len(data_add)

19

In [51]:
data_add[0]

[('Nhân', 'O', 'O', 'O'),
 ('dịp', 'O', 'O', 'O'),
 ('này', 'O', 'O', 'O'),
 (',', 'O', 'O', 'O'),
 ('Fahasa', 'ORGANIZATION', 'ORGANIZATION', 'ORGANIZATION'),
 ('đã', 'O', 'O', 'O'),
 ('tổ', 'O', 'O', 'O'),
 ('chức', 'O', 'O', 'O'),
 ('"', 'O', 'O', 'O'),
 ('Tuần', 'EVENT', 'EVENT', 'EVENT'),
 ('lễ', 'EVENT', 'EVENT', 'EVENT'),
 ('triển', 'EVENT', 'EVENT', 'EVENT'),
 ('lãm', 'EVENT', 'EVENT', 'EVENT'),
 ('sách', 'EVENT', 'EVENT', 'EVENT'),
 ('Nhật', 'EVENT', 'EVENT', 'EVENT'),
 ('Bản', 'EVENT', 'EVENT', 'EVENT'),
 ('tiếng', 'EVENT', 'MISCELLANEOUS', 'MISCELLANEOUS'),
 ('Nhật', 'EVENT', 'EVENT', 'EVENT'),
 ('và', 'EVENT', 'EVENT', 'EVENT'),
 ('khai', 'EVENT', 'EVENT', 'EVENT'),
 ('trương', 'EVENT', 'EVENT', 'EVENT'),
 ('gian', 'EVENT', 'EVENT', 'EVENT'),
 ('hàng', 'EVENT', 'EVENT', 'EVENT'),
 ('"', 'O', 'O', 'O'),
 ('với', 'O', 'O', 'O'),
 ('10.000', 'QUANTITY', 'QUANTITY', 'QUANTITY'),
 ('tên', 'O', 'O', 'O'),
 ('sách', 'O', 'O', 'O'),
 ('tại', 'O', 'O', 'O'),
 ('gian', 'O', 'O', 'O')

In [46]:
data_add[11]

[('Theo', 'O', 'O', 'O'),
 ('tạp', 'O', 'O', 'O'),
 ('chí', 'O', 'O', 'O'),
 ('Vanity', 'ORGANIZATION', 'ORGANIZATION', 'ORGANIZATION'),
 ('Fair', 'ORGANIZATION', 'ORGANIZATION', 'ORGANIZATION'),
 (',', 'O', 'O', 'O'),
 ('hôm', 'DATETIME', 'DATETIME', 'DATETIME'),
 ('2/10/2017', 'DATETIME', 'DATETIME', 'DATETIME'),
 (',', 'O', 'O', 'O'),
 ('Harvey', 'PERSON', 'PERSON', 'PERSON'),
 ('Weinstein', 'PERSON', 'PERSON', 'PERSON'),
 ('đi', 'O', 'O', 'O'),
 ('làm', 'O', 'O', 'O'),
 ('sớm', 'O', 'O', 'O'),
 ('hơn', 'O', 'O', 'O'),
 ('thường', 'O', 'O', 'O'),
 ('lệ', 'O', 'O', 'O'),
 ('.', 'O', 'O', 'O'),
 ('Như', 'O', 'O', 'O'),
 ('mọi', 'O', 'O', 'O'),
 ('ngày', 'O', 'O', 'O'),
 (',', 'O', 'O', 'O'),
 ('ông', 'O', 'O', 'O'),
 ('ta', 'O', 'O', 'O'),
 ('quát', 'O', 'O', 'O'),
 ('tháo', 'O', 'O', 'O'),
 ('nhân', 'O', 'O', 'O'),
 ('viên', 'O', 'O', 'O'),
 ('ầm', 'O', 'O', 'O'),
 ('ĩ', 'O', 'O', 'O'),
 ('trong', 'O', 'O', 'O'),
 ('văn', 'O', 'O', 'O'),
 ('phòng', 'O', 'O', 'O'),
 ('The', 'ORGANIZAT

In [8]:
w, tr_n, pred_n, pos_n = list(zip(*data[2]))

In [10]:
" ".join(w)

'Liên quan đến việc khắc phục hậu quả trong tội cố ý làm trái , dẫn Điều 193 của BLHS , luật sư Hoàng Anh Tuấn nêu quan điểm , trong trường hợp này Vũ Đức Thuận là người nhân danh pháp nhân ( PVC ) thực hiện các giao dịch vì lợi ích của pháp nhân , tức việc chi tiêu tiền là có lợi cho PVC . Với tư cách đó , pháp nhân chịu trách nhiệm với thiệt hại mà người đại diện thực hiện vì pháp nhân gây ra .'

In [6]:
def to_zip(df, parent_path, fold_name):

    compression_opts = dict(method='zip',
                                archive_name=fold_name + '.csv')  
    df.to_csv( parent_path + fold_name + '.zip', index=False,
                compression=compression_opts) 
    return print(fold_name + " saved done!")

In [7]:
import os

def convert_Dataframe(in_folder, out_folder):
    
    sub_folders = [name for name in os.listdir(in_folder) if os.path.isdir(os.path.join(in_folder, name))]
    print(sub_folders)
    for n_folder in sub_folders:
        df = pd.DataFrame(columns = ['f_name','n_sent', 'word', 'label1', 'label2', 'label3'])
        path = in_folder + n_folder
        sub_files = os.listdir(path)
    
  
        for s_file in sub_files:
            try:
                f_path = path + '/' + s_file
                data = read_conll(f_path)
                df1 = pd.DataFrame(data[0])
                df = df.append(df1)
            except  Exception as e: print('error read_conll: {}, {}'.format(e, f_path))

        try:
            to_zip(df, out_folder, n_folder)
        except  Exception as e: print('error to_zip: {}'.format(e))
        


    return 'done!'

        

x

In [8]:
out_folder = '/Users/phamvanmanh/Desktop/machine/final_vlsp/converted_to_dataframe/'
in_folder = '/Users/phamvanmanh/Desktop/machine/final_vlsp/converted/'

In [36]:
convert_Dataframe(in_folder, out_folder)

['test-converted', 'dev-converted', 'train-converted']
test-converted saved done!
dev-converted saved done!
train-converted saved done!


'done!'